In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv("../.env")

In [29]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [13]:
tool = TavilySearchResults(max_results=2)

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# model = ChatOllama(model="llama3.2", temperature=0)

In [30]:
llm = ChatOpenAI(
    api_key="ollama",
    model="llama3.2",
    base_url="http://localhost:11434/v1",
)

In [31]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 26, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-99c01f5b-f4ea-4396-9217-64c36a69cdbc-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

In [32]:
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}

from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(llm, [tool], system=prompt, checkpointer=checkpointer)

In [33]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v["messages"])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9wl0ka1l', 'function': {'arguments': '{"query":"sf weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 258, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6ee89159-0795-49ca-97e3-a1757657395b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': 'call_9wl0ka1l', 'type': 'tool_call'}], usage_metadata={'input_tokens': 258, 'output_tokens': 22, 'total_tokens': 280, 'input_token_details': {}, 'output_token_details': {}})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': 'call_9wl0ka1l', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'h

In [34]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7e33wt1d', 'function': {'arguments': '{"query":"los angeles weather February 2025"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 964, 'total_tokens': 992, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c253e3ac-0d0f-4220-a727-fbf228d08c8f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'los angeles weather February 2025'}, 'id': 'call_7e33wt1d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 964, 'output_tokens': 28, 'total_tokens': 992, 'input_token_details': {}, 'output_token_details': {}})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'los angeles weather February 2025'}, 'id': 'call_7e33

In [35]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nsf20evc', 'function': {'arguments': '{"query":"warmest month in Los Angeles vs San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1919, 'total_tokens': 1948, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5669d124-2612-42be-a112-4dd55cf5dd54-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'warmest month in Los Angeles vs San Francisco'}, 'id': 'call_nsf20evc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1919, 'output_tokens': 29, 'total_tokens': 1948, 'input_token_details': {}, 'output_token_details': {}})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'warmest month in Los Ange

In [36]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9l5l70at', 'function': {'arguments': '{"query":"which one is warmer"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 256, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d96940f7-0d1d-421e-9832-d6143a584124-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'which one is warmer'}, 'id': 'call_9l5l70at', 'type': 'tool_call'}], usage_metadata={'input_tokens': 256, 'output_tokens': 24, 'total_tokens': 280, 'input_token_details': {}, 'output_token_details': {}})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'which one is warmer'}, 'id': 'call_9l5l70at', 'type': 'tool_call'}
Back to the mo